In [16]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

In [60]:
# problem 1
x1 = np.random.uniform(0, 1, 100)
x2 = np.random.uniform(0, 1, 100)
x3 = np.random.uniform(0, 1, 100)
epis = np.random.normal(0, 1, 100)

In [61]:
y = 1 + x1 + 2*x2 + epis

In [62]:
# (1)
x0 = np.ones(100).reshape(-1, 1)
x1 = x1.reshape(-1, 1)
x2 = x2.reshape(-1, 1)
x3 = x3.reshape(-1, 1)
X = np.concatenate((x0, x1, x2, x3), axis=1)
y = y.reshape(-1, 1)

In [63]:
model = sm.OLS(y, X)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.382
Model:                            OLS   Adj. R-squared:                  0.362
Method:                 Least Squares   F-statistic:                     19.75
Date:                Sun, 16 Apr 2023   Prob (F-statistic):           4.71e-10
Time:                        23:54:00   Log-Likelihood:                -145.86
No. Observations:                 100   AIC:                             299.7
Df Residuals:                      96   BIC:                             310.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7393      0.324      2.280      0.025       0.096       1.383
x1             1.1774      0.371      3.171      0.002       0.440       1.914
x2             2.4578      0.374      6.580      0.000       1.716       3.199
x3            -0.3514      0.379     -0.928      0.356      -1.103       0.400
==============================================================================
Omnibus:                        0.711   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.701   Jarque-Bera (JB):                0.758
Skew:                           0.194   Prob(JB):                        0.684
Kurtosis:                       2.823   Cond. No.                         6.01
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [64]:
print("The 95% CI for beta0 is [0.096, 1.383]")
print("The 95% CI for beta1 is [0.440, 1.914]")
print("The 95% CI for beta2 is [1.716, 3.199]")
print("The 95% CI for beta3 is [-1.103, 0.400]")

The 95% CI for beta0 is [0.096, 1.383]
The 95% CI for beta1 is [0.440, 1.914]
The 95% CI for beta2 is [1.716, 3.199]
The 95% CI for beta3 is [-1.103, 0.400]


In [65]:
# (2)
y_hat = 0.7393 + 1.1774*x1 + 2.4578*x2 -0.3514*x3
se = 0
for i in range(y.size):
    se = se + (y[i]-y_hat[i])**2
sr = 0
for i in range(y.size):
    sr = sr + (y_hat[i]-y.mean())**2
f = (sr/3)/(se/96)
fDistribution = stats.f(3, 96)
p_value = 1-fDistribution.cdf(f)
print("The SSE is {}".format(se))
print("The SSR is {}".format(sr))
print("The value for F statistic is {}".format(f))
print("The corresponding p-value is {}".format(p_value))

The SSE is [108.25383717]
The SSR is [66.81679203]
The value for F statistic is [19.75114602]
The corresponding p-value is [4.71339079e-10]


In [33]:
import math

In [55]:
# (3)
def simulate(n):
    x1 = np.random.uniform(0, 1, n).reshape(-1, 1)
    x2 = np.random.uniform(0, 1, n).reshape(-1, 1)
    x3 = np.random.uniform(0, 1, n).reshape(-1, 1)
    x0 = np.ones(n).reshape(-1, 1)
    epis = np.random.normal(0, 1, n).reshape(-1, 1)
    y = 1+x1+2*x2+epis
    X = np.concatenate((x0, x1, x2, x3), axis=1)
    beta_hat = np.linalg.inv((X.T).dot(X)).dot(X.T).dot(y)
    S_square = (((y-X.dot(beta_hat)).T).dot(y-X.dot(beta_hat)))/(n-4)
    d = math.sqrt((S_square * np.linalg.inv((X.T).dot(X)))[-1, -1])
    t = beta_hat[-1]/d
    tDistribution = stats.t(n-4)
    p = 1 - tDistribution.cdf(abs(t)) + tDistribution.cdf((-1)*abs(t))
    return p[0]


In [58]:
count = 0
for i in range(1000):
    p_value = simulate(100)
    if (p_value < 0.05):
        count +=1
type_1_error = count/1000
print("The number of simulation with p-value < 0.05 is about {}".format(count))
print("the empirical type 1 error rate is {} using significance level 0.05".format(type_1_error))

The number of simulation with p-value < 0.05 is about 48
the empirical type 1 error rate is 0.048 using significance level 0.05


In [80]:
# (4)
x_0 = np.array([1, 0.3, 0.2, 0.7]).reshape(-1, 1)
beta_hat = np.linalg.inv((X.T).dot(X)).dot(X.T).dot(y)
y0_hat = (x_0.T).dot(beta_hat)[0][0]
S_square = (((y-y_hat).T).dot(y-y_hat))/96
e = 1+(x_0.T).dot(np.linalg.inv((X.T).dot(X))).dot(x_0)
d = math.sqrt(S_square*e)
tDistribution = stats.t(96)
lb = y0_hat-tDistribution.interval(0.95)[1]*d
ub = y0_hat+tDistribution.interval(0.95)[1]*d
print("The predicted value for y0 is {}".format(y0_hat))
print("The prediction interval for y0 is [{}, {}]".format(lb, ub))

The predicted value for y0 is 1.3380341338004342
The prediction interval for y0 is [-0.7992041375842054, 3.475272405185074]


In [83]:
# (5)
def simul(n):
    x1 = np.random.uniform(0, 1, n).reshape(-1, 1)
    x2 = np.random.uniform(0, 1, n).reshape(-1, 1)
    x3 = np.random.uniform(0, 1, n).reshape(-1, 1)
    x0 = np.ones(n).reshape(-1, 1)
    epis = np.random.normal(0, 1, n).reshape(-1, 1)
    y = 1+x1+2*x2+epis
    X = np.concatenate((x0, x1, x2, x3), axis=1)
    beta_hat = np.linalg.inv((X.T).dot(X)).dot(X.T).dot(y)
    y_hat = X.dot(beta_hat)
    x_0 = np.array([1, 0.3, 0.2, 0.7]).reshape(-1, 1)
    y0_hat = (x_0.T).dot(beta_hat)[0][0]
    S_square = (((y-y_hat).T).dot(y-y_hat))/(n-4)
    e = 1+(x_0.T).dot(np.linalg.inv((X.T).dot(X))).dot(x_0)
    d = math.sqrt(S_square*e)
    t1 = stats.t(n-4)
    lb = y0_hat-t1.interval(0.95)[1]*d
    ub = y0_hat+t1.interval(0.95)[1]*d
    return ub-lb

In [84]:
width_1 = simul(200)
width_2 = simul(500)
width_3 = simul(1000)
print("The width of 95% prediction intervals for y0 are {}, {}, {} respectively for n=200, 500, 1000".format(width_1, width_2, width_3))

The width of 95% prediction intervals for y0 are 3.6564660410395122, 3.798808441126975, 3.989859305477668 respectively for n=200, 500, 1000


The width does not approach to zero, but diverges gradually

In [85]:
# Problem 2
data = pd.read_csv("E://sample/traffic.csv")
data

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012/10/2 9:00,5545
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012/10/2 10:00,4516
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012/10/2 11:00,4767
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012/10/2 12:00,5026
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012/10/2 13:00,4918
...,...,...,...,...,...,...,...,...,...
48199,None,283.45,0.0,0.0,75,Clouds,broken clouds,2018/9/30 19:00,3543
48200,None,282.76,0.0,0.0,90,Clouds,overcast clouds,2018/9/30 20:00,2781
48201,None,282.73,0.0,0.0,90,Thunderstorm,proximity thunderstorm,2018/9/30 21:00,2159
48202,None,282.09,0.0,0.0,90,Clouds,overcast clouds,2018/9/30 22:00,1450


In [88]:
# figure out the three treatment groups
data1 = data[data['weather_main'] == 'Clouds']
data2 = data[data['weather_main'] == 'Clear']
data3 = data[data['weather_main'] == 'Snow']

In [91]:
data1 = data1.reset_index()
data1 = data1.drop(columns = 'index')
data1 = data1.drop(columns = 'level_0')
data1

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012/10/2 9:00,5545
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012/10/2 10:00,4516
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012/10/2 11:00,4767
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012/10/2 12:00,5026
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012/10/2 13:00,4918
...,...,...,...,...,...,...,...,...,...
15159,None,284.79,0.0,0.0,75,Clouds,broken clouds,2018/9/30 17:00,4132
15160,None,283.45,0.0,0.0,75,Clouds,broken clouds,2018/9/30 19:00,3543
15161,None,282.76,0.0,0.0,90,Clouds,overcast clouds,2018/9/30 20:00,2781
15162,None,282.09,0.0,0.0,90,Clouds,overcast clouds,2018/9/30 22:00,1450


In [95]:
data2 = data2.reset_index()
data2 = data2.drop(columns = 'index')
data2 = data2.drop(columns = 'level_0')
data2

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,291.72,0.0,0.0,1,Clear,sky is clear,2012/10/2 14:00,5181
1,None,293.17,0.0,0.0,1,Clear,sky is clear,2012/10/2 15:00,5584
2,None,293.86,0.0,0.0,1,Clear,sky is clear,2012/10/2 16:00,6015
3,None,289.38,0.0,0.0,1,Clear,sky is clear,2012/10/2 20:00,2784
4,None,288.61,0.0,0.0,1,Clear,sky is clear,2012/10/2 21:00,2361
...,...,...,...,...,...,...,...,...,...
13386,None,274.25,0.0,0.0,1,Clear,sky is clear,2018/9/29 4:00,425
13387,None,274.66,0.0,0.0,1,Clear,sky is clear,2018/9/29 5:00,743
13388,None,274.62,0.0,0.0,1,Clear,sky is clear,2018/9/29 6:00,1359
13389,None,274.79,0.0,0.0,1,Clear,sky is clear,2018/9/29 7:00,2036


In [98]:
data3 = data3.reset_index()
data3 = data3.drop(columns = 'index')
data3 = data3.drop(columns = 'level_0')
data3

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,275.66,0.0,0.0,90,Snow,heavy snow,2012/10/26 9:00,5234
1,None,275.02,0.0,0.0,90,Snow,heavy snow,2012/10/26 10:00,4695
2,None,275.03,0.0,0.0,90,Snow,heavy snow,2012/10/26 11:00,5341
3,None,274.50,0.0,0.0,90,Snow,heavy snow,2012/10/26 12:00,5943
4,None,275.01,0.0,0.0,90,Snow,heavy snow,2012/10/26 13:00,5874
...,...,...,...,...,...,...,...,...,...
2871,None,272.01,0.0,0.0,90,Snow,light snow,2018/4/16 12:00,4149
2872,None,274.23,0.0,0.0,90,Snow,light snow,2018/4/18 11:00,4824
2873,None,274.37,0.0,0.0,90,Snow,light snow,2018/4/18 12:00,5013
2874,None,276.23,0.0,0.0,90,Snow,light snow,2018/4/18 16:00,7022


In [104]:
# calculate SS(T) and SSE
summ = data1['traffic_volume'].sum()+data2['traffic_volume'].sum()+data3['traffic_volume'].sum()
de = data1['traffic_volume'].size+data2['traffic_volume'].size+data3['traffic_volume'].size
total_mean = summ/de
s1 = data1['traffic_volume'].size * (data1['traffic_volume'].mean()-total_mean)**2
s2 = data2['traffic_volume'].size * (data2['traffic_volume'].mean()-total_mean)**2
s3 = data3['traffic_volume'].size * (data3['traffic_volume'].mean()-total_mean)**2
sst = s1+s2+s3
t1 = 0
t2 = 0
t3 = 0
for i in range(data1['traffic_volume'].size):
    t1 += (data1['traffic_volume'][i]-data1['traffic_volume'].mean())**2
for i in range(data2['traffic_volume'].size):
    t2 += (data2['traffic_volume'][i]-data2['traffic_volume'].mean())**2
for i in range(data3['traffic_volume'].size):
    t3 += (data3['traffic_volume'][i]-data3['traffic_volume'].mean())**2
sse = t1+t2+t3
tss = sst+sse
dof = de-3
ms1 = sst/2
ms2 = sse/dof
f_value = ms1/ms2

In [105]:
df = pd.DataFrame({'SS':{'Treatment':sst, 'Error':sse, 'Total':tss}, 
'df':{'Treatment': 2, 'Error': dof, 'Total': dof+2}, 'MS':{'Treatment': ms1, 'Error':ms2}, 
'F':{'Treatment': f_value}})
df

,SS,df,MS,F
Treatment,2.548512e+09,2,1.274256e+09,338.384375
Error,1.183486e+11,31428,3.765705e+06,NaN
Total,1.208971e+11,31430,NaN,NaN


The degree of freedom I use in this F-test is 2, 31428

In [106]:
f1 = stats.f(2, 31428)
f1.isf(0.05)

2.9960178463550626

The F value that I calculated is 338.38, and reference value of significance level 0.05 is 2.996, so I will reject the null hypothesis, that is the hourly traffic has different means under these 3 weather conditions